# Submitted by:
 
* Name 1: Adam Ginton
* Name 2: Or Lior
* Name 3: Sharon Edri

## To be submitted by: Unknown yet

# paint_the_circle Project

This project identifies and paints circles given in RGB images. Specifically, each input image contains a rectangle, a triangle and a circle. The image is fed into a neural netork, whose outputs are the center and radius of the circle within the image. The domain considered is "[0,1]x[0,1] "box". PyTorch is the platform used. The goal is to paint the circle.

### **You are given the following files:**
  Three directories named "train", "validation", and "test" each containing the following:
  1. a directory called "images" with RGB png files of size 128x128 (so image that is read has shape (3,128,128))
  2. a file "labels.txt" containing the center points (as x,y) and radius of the circles for all images in "images" directory

Since this problem is somewhat open ended, you are given a suggested structure and some functionality that is already implemented for you (see below). You are not at all obigated to follow this structure or use the code given. 

**Note:** 
- "network" and "model" are used interchangeably throughout this documentation.
 
 
### **General structure (in order):**
 1. DataSet and DataLoader            - this is how you load data into the network **(given to you)**
 2. Neural Network definition         - the class that defines your model   **(need to implement)**
 3. calculate model parameters number - useful to get a feel for network's size **(given to you)**
 4. loss function definition          - to be used in training **(need to implement)**
 5. create an optimizer               - choose your optimizer **(need to implement)**
 6. estimate number of ops per forward feed - the bigger this is the slower the training **(given to you)**
 7. estimate ops per forward feed     - use function given to you to estimate this **(implement for convenience)**
 8. view images and target and net labels - example code to help you see how to use loaders **(given to you)**
 9. validate_model                   - returns avg loss per image for a model and loader  **(need to implement)**
 10. train_model                      - trains the network **(need to implement)**
 11. plot train/validaion losses      - visualizing train and validation losses from training **(given to you)**
 12. save/load model                  - Allows you to save and load model to disk for later use **(given to you)**
 13. visualizing images    - Painting circles prodiced by network on images from a given loader **(given to you)**
 
 
### ** What you need to do **
You have 5 (and a half...) things to do:
 1. Create a CircleNet class, which is your network model. This is a key component. The output of your model should be 3 numbers that represent the the center and radius of the circle in the input image fed into your network. You should keep in mind the number of parameters in your model. If there are too many, it may overfit (and take longer to run). If there are too few, you it may not be able to learn the task needed. A typical structure would have convolutional layers first and fully connected at the end, thus reducing number of parameteres. Consider which activation function you want to use, and whether or not you wish to use batch normalization or dropout. Pooling layers are also possible. Be creative.
 
 2. Create a loss function. This is another key component as it defines what it means for two circles (the true circle and its estimation) to be similar or not. Namely, two circles (represented by two centers and two radii: ground truth ("labels") and network outputs ("outputs")), whose images look similar should have a smaller loss than two circles whose images look less simialr. Think about how you would quantify "closeness"/"similarity" of circles.

 3. Choose an optimizer. Look here for some ideas: https://pytorch.org/docs/master/optim.html
 
 4. (This is the "half" thing to do). For your conveinece you may want to use the function calc_ops() that is given to you to calculate an estimate of the number of operations that your network does per feed forward. To do this, you need to enter your own network structure. An example of an arbitrary network is privided to you.
 
 5. Create a validate_model function that assesses (tests) the performance of a model. It returns the average loss per image for a given loader. It can be run on any set of data (train, validation, or test). You may want to run this function on validation set (loader) from within the train function (see below) train after each epoch so as to see how loss behaves on validation during the training process.
 
 6. Create a train_model function that trains model. This function updates the following parameteres: model, train_losses, and validation_losses. Model is updated simply by the training processes when optimzer.step() is called. The other two parameteres are lists that hold the average loss per image for the corresponding data (train or validation). After every epoch (i.e., iteration that goes over the entire train data), you should save to these lists the average loss per image for the corresponding data loader. These lists are useful in that you can plot them (functionality given to you) and observe how model behaves. Observe: this fucnton returs nothing, however, it updates parameteres by reference. Specifically, model is updated (trained), and so are train and validation losses values.
 
**Note:** You are given three sets of data: trian, validation, and test. It is recommended that test not be touched until the very end, and validation be used to get a sense of your network's performance. 

### ** Running on a GPU: **
It is not necessary to use a GPU for this project. However, if you choose to do so, you will gain a major speedup to your training, which will save you much time. The code given to you identifies the hardware used and will  automatically run on either a GPU or CPU. 

### ** Useful links: **
1. PyTorch master tutorial - VERY useful: https://pytorch.org/docs/master/nn.html
2. PyTorch optimizers: https://pytorch.org/docs/master/optim.html
3. A list of possible reasons why things go wrong: https://blog.slavv.com/37-reasons-why-your-neural-network-is-not-working-4020854bd607#74de

### ** Final tips: **
Use the Internet! Things will not work first time. You will get strange error messages. Google them up. The web is  great resource for tackling problems ranging from python error messages, to things not doing what you'd like them to do.


### ** Submission Instructions**
The project is to be submittd in teams as in the homework. You need to submit the following three files:
1. model.dat                            - This is your saved model 
2. paint_that_circle.ipynb              - This is this notebook containing all of your work
3. model.py                             - A file containing your CircleNet class only. 

Before you submit, run "check_before_submission.ipynb" to make sure your model could be properly tested. See instructions for running this notebook inside.

Make sure your names appear at the top of this notebook in the appropriate place.

### **GOOD LUCK!**

In [1]:
import numpy as np
import PIL.Image as Image
from PIL import ImageDraw


import torch.nn as nn

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os
import glob
import datetime

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Dataset class definition (given to you) 

In [2]:
class ShapesDataset(Dataset):
    
    def __init__(self, dataset_dir):
        """
        Initializing dataset by generating a dicitonary of labels, where an image file name is the key 
        and its labels are the contents of that entry in the dictionary. Images are not loaded. This way it
        is possible to iterate over arbitrarily large datasets (limited by labels dicitonary fitting 
        in memory, which is not a problem in practice)
        
        Args:
            dataset_dir : path to directory with images and labels. In this directory we expect to find
                          a directory called "images" containing the input images, and a file called 
                          "labels.txt" containing desired labels (coefficients)
        """
        
        self.dataset_dir = dataset_dir
        self.labels_dict = self.gen_labels_dict()
        self.images_keys = list(self.labels_dict)  # getting the keys of the dictionary as list
        self.images_keys.sort()                    # sorting so as to have in alphabetical order 

    def __len__(self):
        return len(self.labels_dict)

    def __getitem__(self, index):    
        """
        This funtion makes it possible to iterate over the ShapesDataset
        Args:
            index: running index of images
            
        Returns:
            sample: a dicitionary with three entries:
                    1. 'image'  contains the image
                    2. 'labels' contains labels (coeffs) corresponding to image
                    3. 'fname'  contains name of file (image_key) - may be useful for debugging
        """
        image_key = self.images_keys[index]     # recall - key is the file name of the corresponding image
        image = np.array(Image.open(image_key)) # image has shape: (128, 128, 3)
        image = image/255.0                     # simple normalization - just to maintain small numbers
        image = np.transpose(image, (2, 0, 1))  # network needs RGB channels to be first index
        labels = self.labels_dict[image_key]
        sample = {'image': image, 'labels': labels, 'fname':image_key}
        
        #return(image, labels, image_key)
        return sample
    
    
    def gen_labels_dict(self):
        """
        This fucntion generates a dictionary of labels
        
        Returns:
            labels_dict: the key is image file name and the value is the corresponding 
            array of labels  
        """
        
        labels_fname = self.dataset_dir + "/labels.txt"
        labels_dict = {}
        with open(labels_fname, "r") as inp:
            for line in inp:
                line = line.split('\n')[0]                                      # remove '\n' from end of line 
                line = line.split(',')
                key  = self.dataset_dir + '/images/' + line[0].strip() + ".png" # image file name is the key
                del line[0]
                
                list_from_line = [float(item) for item in line]
                labels_dict[key] = np.asarray(list_from_line, dtype=np.float32)
                        
        return labels_dict             


# Create Data Loaders (given to you) 

In [3]:
train_dir      = "./train/"
validation_dir = "./validation/"
test_dir       = "./test/"


train_dataset = ShapesDataset(train_dir)

train_loader = DataLoader(train_dataset, 
                          batch_size=32,
                          shuffle=True)



validation_dataset = ShapesDataset(validation_dir)

validation_loader = DataLoader(validation_dataset, 
                               batch_size=1,
                               shuffle=False)



test_dataset = ShapesDataset(test_dir)

test_loader = DataLoader(test_dataset, 
                          batch_size=1,
                          shuffle=False)


print("train loader examples     :", len(train_dataset)) 
print("validation loader examples:", len(validation_dataset))
print("test loader examples      :", len(test_dataset))

train loader examples     : 1000
validation loader examples: 100
test loader examples      : 100


# Neural Network class definition

In [4]:
import torch.nn.functional as F  # a lower level (compared to torch.nn) interface
import torchvision.transforms as transforms


class CircleNet(nn.Module):    # nn.Module is parent class  
    def __init__(self):
        super(CircleNet, self).__init__()  #calls init of parent class
                

        #----------------------------------------------
        # implementation needed here 
        #----------------------------------------------
      
        
        # The convolution layers were chosen to keep dimensions of input image: (I-F+2P)/S +1= (128-3+2)/1 + 1 = 128
        
        # Our images are RGB, so input channels = 3. Use 12 filters for first 2 convolution layers, then double
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=24, out_channels=32, kernel_size=3, stride=1, padding=1)

        
        
        #Pooling to reduce sizes, and dropout to prevent overfitting
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        
        self.drop = nn.Dropout2d(p=0.2)
#         self.norm1 = nn.BatchNorm2d(12)
#         self.norm2 = nn.BatchNorm2d(24)
        
        # There are 2 pooling layers, each with kernel size of 2. Output size: 128/(2*2) = 32
        # Feature tensors are now 32 x 32; With 24 output channels, this gives 32x32x24
        
        # Have 3 output features, corresponding to x-pos, y-pos, radius. 
        self.fc = nn.Linear(in_features=32 * 32 * 32, out_features=3)
    

                
    def forward(self, x):
        """
        Feed forward through network
        Args:
            x - input to the network
            
        Returns "out", which is the network's output
        """
        
        #----------------------------------------------
        # implementation needed here 
        #----------------------------------------------
        
        #Convolution 1
        out = self.conv1(x)
        out = self.relu(out)
        out = self.pool(out)
        
#         out = self.norm1(out)

        #Convolution 2
        out = self.conv2(out)
        out = self.relu(out)
        out = self.pool(out)
        
#         out = self.norm1(out)
        
        #Convolution 3
        out = self.conv3(out)
        out = self.relu(out)
        out = self.drop(out)
#         out = self.norm2(out)
        
        #Convolution 4
        out = self.conv4(out)
        out = self.relu(out)
        out = F.dropout(out, training=self.training)
        
        
        out = out.view(-1, 32 * 32 * 32)
        out = self.fc(out)

        
        return out
             
             


# Loss function definition

In [5]:
def my_loss(outputs, labels):
    
    """
    Args:
        outputs - output of network ([batch size, 3]) 
        labels  - desired labels  ([batch size, 3])
    """
    
    loss = torch.zeros(1, dtype=torch.float, requires_grad=True)
    loss = loss.to(device)
    
    #----------------------------------------------
    # implementation needed here 
    #----------------------------------------------

    loss = criterion(outputs, labels)
    
    
    
    
    
    # Observe: If you need to iterate and add certain values to loss defined above
    # you cannot write: loss +=... because this will raise the error: 
    # "Leaf variable was used in an inplace operation"
    # Instead, to avoid this error write: loss = loss + ...  
    
                                      
    return loss


# Get number of trainable parameters (given to you)  

In [6]:
def get_train_params_num(model):
    """
    This fucntion returns the number of trainable parameters of neural network model
    You may want to call it after you create your model to see how many parameteres the model has
    Args:
        model - neural net to examine
    """
    
    #filter given iterable with a function that tests each element in the iterable to be true or not
    model_parameters = filter(lambda p: p.requires_grad == True, model.parameters()) 
    params_num = sum([np.prod(p.size()) for p in model_parameters])
    return params_num


# Model creation and choice of optimizer

In [7]:
model = CircleNet().to(device)
print ("Number of model trainable parameters:", get_train_params_num(model))

print()
print("Model:")
print(model)  #Used to see model to more easily plug into calc_ops()



#----------------------------------------------
#  Choose your optimizer:
#  implementation needed here 
#----------------------------------------------
# Use an "Adam" optimizer to adjust weights
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay =0.001)



####
criterion = nn.MSELoss()


Number of model trainable parameters: 109511

Model:
CircleNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=32768, out_features=3, bias=True)
)


# Get an estimate number of operations (given to you)

In [8]:
def calc_ops(inp_size, net_struct):
    """
    Calculates a rough number of operations for a given network topology
    Args:
        inp_size - (W,H) of input 
        net_struct - list of tuples describing structure of network. 
        
        Example:
         (('conv2d', (3, 8, 3, 1, 0)),  # cin, cout, kernel, stride, pad
          ('conv2d': 83, 8, 3, 1, 0)),
          ('MaxPool2d', (2,2)),         # kernel, stride
          ('fc': (64, 8)),      
          ('fc': (8, 4)))
         
    """
    
    ops = 0
    W, H = inp_size
    for curr_item in net_struct:
        if curr_item[0] == 'conv2d':
            cin = curr_item[1][0]
            cout = curr_item[1][1]
            kernel = curr_item[1][2]
            stride = curr_item[1][3]
            pad = curr_item[1][4]
            W = (W +2*pad - kernel)/stride + 1
            H = (H +2*pad - kernel)/stride + 1
            curr_ops = (W*H*cin*cout*kernel*kernel)/stride
            ops += curr_ops
            print (curr_item, ":",  "{:,}".format(int(curr_ops)))
        elif curr_item[0] == 'MaxPool2d':
            kernel = curr_item[1][0]
            stride = curr_item[1][1]
            W = (W - kernel)/stride + 1
            H = (H - kernel)/stride + 1
        else:
            curr_ops = curr_item[1][0] * curr_item[1][1]
            ops += curr_ops
            print (curr_item, ":",  "{:,}".format(int(curr_ops)))
            
    return int(ops)

# Check rough number of ops for network (for your convenience)

In [9]:
inp_size = (128,128)

# place your network ropology in example_net below to obtain an estimated number of operations for your network
example_net = (('conv2d', (3, 3, 3, 1, 1)),
               ('MaxPool2d', (2,2)),
               ('conv2d', (3, 3, 3, 1, 1)),
               ('MaxPool2d', (2,2)),
               ('fc', (2883, 4)))


circlenet1 = (('conv1', (3,12,3,1,1)),
          ('conv2',(12,12,3,1,1)),
          ('conv3', (12,24,3,1,1)),
          ('pool', (2,2)), 
          ('fc', (24576,3)))



ops = calc_ops(inp_size, example_net)
print()
print("Example Total ops: {:,}".format(ops))

num_ops_circlenet1 = calc_ops(inp_size, circlenet1)
print()
print("CircleNet Total ops: {:,}".format(num_ops_circlenet1))


('conv2d', (3, 3, 3, 1, 1)) : 1,327,104
('conv2d', (3, 3, 3, 1, 1)) : 331,776
('fc', (2883, 4)) : 11,532

Example Total ops: 1,670,412
('conv1', (3, 12, 3, 1, 1)) : 36
('conv2', (12, 12, 3, 1, 1)) : 144
('conv3', (12, 24, 3, 1, 1)) : 288
('pool', (2, 2)) : 4
('fc', (24576, 3)) : 73,728

CircleNet Total ops: 74,200


# View images, target circle labels and  network outputs (given to you)

In [10]:
"""
View first image of a given number of batches assuming that model has been created. 
Currently, lines assuming model has been creatd, are commented out. Without a model, 
you can view target labels and the corresponding images.
This is given to you so that you may see how loaders and model can be used. 
"""

loader = train_loader # choose from which loader to show images
bacthes_to_show = 2
with torch.no_grad():
    for i, data in enumerate(loader, 0): #0 means that counting starts at zero
        inputs = (data['image']).to(device)   # has shape (batch_size, 3, 128, 128)
        labels = (data['labels']).to(device)  # has shape (batch_size, 3)
        img_fnames = data['fname']            # list of length batch_size
        
        #outputs = model(inputs.float())
        img = Image.open(img_fnames[0])
        
        print ("showing image: ", img_fnames[0])
        
        labels_str = [ float(("{0:.2f}".format(x))) for x in labels[0]]#labels_np_arr]
        
        #outputs_np_arr = outputs[0] # using ".numpy()" to convert tensor to numpy array
        #outputs_str = [ float(("{0:.2f}".format(x))) for x in outputs_np_arr]
        print("Target labels :", labels_str )
        #print("network coeffs:", outputs_str)
        print()
        #img.show()
        
        if (i+1) == bacthes_to_show:
            break
        

showing image:  ./train//images/0828.png
Target labels : [0.38, 0.81, 0.18]

showing image:  ./train//images/0956.png
Target labels : [0.52, 0.32, 0.2]



# Validate model function

In [11]:
def validate_model(model, loader):
    """
    This function parses a given loader and returns the avergae (per image) loss 
    (as defined by "my_loss") of the entire dataset associated with the given loader.
    
    Args:
        model  - neural network to examine
        loader - where input data comes from (train, validation, or test)
        
    returns:
        average loss per image in variable named "avg_loss"
    """

    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
                  # (dropout is set to zero)

    #----------------------------------------------
    # implementation needed here 
    #----------------------------------------------
  
    # return average loss for the epoch
    # Switch the model to evaluation mode (so we don't backpropagate or drop)
    model.eval()
#     validation_loss = 0
    total_validation_loss = 0
    
    with torch.no_grad():
        for batch_id, data in enumerate(loader):
            inputs = (data['image']).to(device)   # has shape (batch_size, 3, 128, 128)
            labels = (data['labels']).to(device)  # has shape (batch_size, 3)
            img_fnames = data['fname']            # list of length batch_size
            
#             data, target = data.to(device), target.to(device)
            output = model(inputs.float())
            # loss for current batch
            loss = my_loss(output, labels)
            total_validation_loss = total_validation_loss + loss.item()
            
            
            # Calculate the loss for this batch and add to running total
#             validation_loss += my_loss(output, labels).item()
            

        avg_loss = total_validation_loss / (batch_id+1)
    
    model.train()  #back to default
    return avg_loss


# train model function

In [12]:
def train_model(model,
                optimizer,
                train_loader,
                validation_loader,
                train_losses,
                validation_losses,
                epochs=1):
    
    """
    Trains a neural network. 
    Args:
        model               - model to be trained
        optimizer           - optimizer used for training
        train_loader        - loader from which data for training comes 
        validation_loader   - loader from which data for validation comes (maybe at the end, you use test_loader)
        train_losses        - adding train loss value to this list for future analysis
        validation_losses   - adding validation loss value to this list for future analysis
        epochs              - number of runs over the entire data set 
    """
    
    #----------------------------------------------
    # implementation needed here 
    #----------------------------------------------
  
    #train_loss = 0
    # model.train()  # set to training mode
    
    print("Epoch:", epochs)
    for ep in range(epochs):      #each epoch loops over entire dataset
        print("\nEpoch # ", ep+1)
        total_training_loss = 0    #RESET THE LOSS FOR NEW EPOCH!!!!
        for batch_id, data in enumerate(train_loader, 0): #0 means that counting starts at zero
            inputs = (data['image']).to(device)   # has shape (batch_size, 3, 128, 128)
            labels = (data['labels']).to(device)  # has shape (batch_size, 3)
            img_fnames = data['fname']            # list of length batch_size   
            

            #reset the optimizer
            optimizer.zero_grad()
            
            #pass data through model
            output = model(inputs.float())
            
            # get loss for current batch
            loss = my_loss(output, labels)
            # Backpropagate
            loss.backward()    
            optimizer.step()
            
            #train_loss is a running total of loss over all batches so far
            total_training_loss = total_training_loss + loss.item()
            
            
            
            images_counted = ((batch_id+1) * 32)
            print('\tTraining batch {}, Images: {}/1000 Total_loss: {}, Loss: {:.6f}'.format(batch_id + 1,images_counted, total_training_loss, loss.item()))
            
        
        #average loss for each batch: 
        avg_loss = total_training_loss / (batch_id+1)
        train_losses.append(avg_loss)   
            
        print("Average loss for Training set {}: {:.6f}".format(ep+1,avg_loss))
        validation_avg_loss = validate_model(model, validation_loader)
        print("Average loss for Validation set {}: {:.6f}".format(ep+1,validation_avg_loss))
        validation_losses.append(validation_avg_loss)
    

    return 
    


# Actual train (given to you)

In [13]:
# Using two lists (train_losses, validation_losses) containing history of losses 
# (i.e., loss for each training epoch) for train and validation sets. 
# If thess are not defined, we define them. Otherwise, the function train_model
# updates these two lists (by adding loss values when it is called for further training) 
# in order to be able to visualize train and validation losses


if not 'train_losses' in vars():
    train_losses = []
if not 'validation_losses' in vars():
    validation_losses = []


train_model(model, 
            optimizer,
            train_loader,   #train_loader, 
            validation_loader, 
            train_losses, 
            validation_losses,
            epochs=30)


Epoch: 30

Epoch #  1
	Training batch 1, Images: 32/1000 Total_loss: 0.20647895336151123, Loss: 0.206479
	Training batch 2, Images: 64/1000 Total_loss: 1.4576423168182373, Loss: 1.251163
	Training batch 3, Images: 96/1000 Total_loss: 1.512425683438778, Loss: 0.054783
	Training batch 4, Images: 128/1000 Total_loss: 1.6254053339362144, Loss: 0.112980
	Training batch 5, Images: 160/1000 Total_loss: 1.8841088935732841, Loss: 0.258704
	Training batch 6, Images: 192/1000 Total_loss: 2.169802747666836, Loss: 0.285694
	Training batch 7, Images: 224/1000 Total_loss: 2.3352193757891655, Loss: 0.165417
	Training batch 8, Images: 256/1000 Total_loss: 2.4324485138058662, Loss: 0.097229
	Training batch 9, Images: 288/1000 Total_loss: 2.473510518670082, Loss: 0.041062
	Training batch 10, Images: 320/1000 Total_loss: 2.5133952647447586, Loss: 0.039885
	Training batch 11, Images: 352/1000 Total_loss: 2.596932530403137, Loss: 0.083537
	Training batch 12, Images: 384/1000 Total_loss: 2.697964869439602, L

	Training batch 29, Images: 928/1000 Total_loss: 0.6114508863538504, Loss: 0.022890
	Training batch 30, Images: 960/1000 Total_loss: 0.6309663262218237, Loss: 0.019515
	Training batch 31, Images: 992/1000 Total_loss: 0.6522771064192057, Loss: 0.021311
	Training batch 32, Images: 1024/1000 Total_loss: 0.6677866149693727, Loss: 0.015510
Average loss for Training set 6: 0.020868
Average loss for Validation set 6: 0.019956

Epoch #  7
	Training batch 1, Images: 32/1000 Total_loss: 0.016189124435186386, Loss: 0.016189
	Training batch 2, Images: 64/1000 Total_loss: 0.03733770735561848, Loss: 0.021149
	Training batch 3, Images: 96/1000 Total_loss: 0.05531678907573223, Loss: 0.017979
	Training batch 4, Images: 128/1000 Total_loss: 0.07760589011013508, Loss: 0.022289
	Training batch 5, Images: 160/1000 Total_loss: 0.09972196631133556, Loss: 0.022116
	Training batch 6, Images: 192/1000 Total_loss: 0.12434652633965015, Loss: 0.024625
	Training batch 7, Images: 224/1000 Total_loss: 0.1476800460368

	Training batch 23, Images: 736/1000 Total_loss: 0.20230083912611008, Loss: 0.009706
	Training batch 24, Images: 768/1000 Total_loss: 0.2082164823077619, Loss: 0.005916
	Training batch 25, Images: 800/1000 Total_loss: 0.21796782268211246, Loss: 0.009751
	Training batch 26, Images: 832/1000 Total_loss: 0.2274006656371057, Loss: 0.009433
	Training batch 27, Images: 864/1000 Total_loss: 0.23387978365644813, Loss: 0.006479
	Training batch 28, Images: 896/1000 Total_loss: 0.2427820940501988, Loss: 0.008902
	Training batch 29, Images: 928/1000 Total_loss: 0.25151212560012937, Loss: 0.008730
	Training batch 30, Images: 960/1000 Total_loss: 0.26041090162470937, Loss: 0.008899
	Training batch 31, Images: 992/1000 Total_loss: 0.26864172192290425, Loss: 0.008231
	Training batch 32, Images: 1024/1000 Total_loss: 0.27775593707337976, Loss: 0.009114
Average loss for Training set 12: 0.008680
Average loss for Validation set 12: 0.007252

Epoch #  13
	Training batch 1, Images: 32/1000 Total_loss: 0.00

	Training batch 17, Images: 544/1000 Total_loss: 0.06624690559692681, Loss: 0.002706
	Training batch 18, Images: 576/1000 Total_loss: 0.07085129641927779, Loss: 0.004604
	Training batch 19, Images: 608/1000 Total_loss: 0.07477398007176816, Loss: 0.003923
	Training batch 20, Images: 640/1000 Total_loss: 0.07818740280345082, Loss: 0.003413
	Training batch 21, Images: 672/1000 Total_loss: 0.08102274313569069, Loss: 0.002835
	Training batch 22, Images: 704/1000 Total_loss: 0.0836071704979986, Loss: 0.002584
	Training batch 23, Images: 736/1000 Total_loss: 0.08823954849503934, Loss: 0.004632
	Training batch 24, Images: 768/1000 Total_loss: 0.09249612432904541, Loss: 0.004257
	Training batch 25, Images: 800/1000 Total_loss: 0.09611106407828629, Loss: 0.003615
	Training batch 26, Images: 832/1000 Total_loss: 0.09960851073265076, Loss: 0.003497
	Training batch 27, Images: 864/1000 Total_loss: 0.10282719926908612, Loss: 0.003219
	Training batch 28, Images: 896/1000 Total_loss: 0.106755309272557

	Training batch 10, Images: 320/1000 Total_loss: 0.029740408761426806, Loss: 0.003023
	Training batch 11, Images: 352/1000 Total_loss: 0.03304095147177577, Loss: 0.003301
	Training batch 12, Images: 384/1000 Total_loss: 0.03577021788805723, Loss: 0.002729
	Training batch 13, Images: 416/1000 Total_loss: 0.03861308703199029, Loss: 0.002843
	Training batch 14, Images: 448/1000 Total_loss: 0.04224020289257169, Loss: 0.003627
	Training batch 15, Images: 480/1000 Total_loss: 0.045343044213950634, Loss: 0.003103
	Training batch 16, Images: 512/1000 Total_loss: 0.04821932315826416, Loss: 0.002876
	Training batch 17, Images: 544/1000 Total_loss: 0.05187062150798738, Loss: 0.003651
	Training batch 18, Images: 576/1000 Total_loss: 0.053844320587813854, Loss: 0.001974
	Training batch 19, Images: 608/1000 Total_loss: 0.056559166638180614, Loss: 0.002715
	Training batch 20, Images: 640/1000 Total_loss: 0.059045698028057814, Loss: 0.002487
	Training batch 21, Images: 672/1000 Total_loss: 0.061841385

	Training batch 2, Images: 64/1000 Total_loss: 0.0047929673455655575, Loss: 0.002417
	Training batch 3, Images: 96/1000 Total_loss: 0.007698295172303915, Loss: 0.002905
	Training batch 4, Images: 128/1000 Total_loss: 0.010532640852034092, Loss: 0.002834
	Training batch 5, Images: 160/1000 Total_loss: 0.013260368956252933, Loss: 0.002728
	Training batch 6, Images: 192/1000 Total_loss: 0.015571783995255828, Loss: 0.002311
	Training batch 7, Images: 224/1000 Total_loss: 0.01811851537786424, Loss: 0.002547
	Training batch 8, Images: 256/1000 Total_loss: 0.020951258251443505, Loss: 0.002833
	Training batch 9, Images: 288/1000 Total_loss: 0.02315811556763947, Loss: 0.002207
	Training batch 10, Images: 320/1000 Total_loss: 0.026354010915383697, Loss: 0.003196
	Training batch 11, Images: 352/1000 Total_loss: 0.029310124227777123, Loss: 0.002956
	Training batch 12, Images: 384/1000 Total_loss: 0.03200458851642907, Loss: 0.002694
	Training batch 13, Images: 416/1000 Total_loss: 0.034290359122678

# Plot losses from training process (given to you)

In [14]:
import matplotlib.pyplot as plt

print("training_loss len", len(train_losses))
print("Type training_loss", type(train_losses))
print(train_losses)

print("validation_losses len", len(validation_losses))
print(validation_losses)

iteration = np.arange(0., len(train_losses))
plt.plot(iteration, train_losses, 'g-', iteration, validation_losses, 'r-')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()
    

training_loss len 30
Type training_loss <class 'list'>
[0.10691430664155632, 0.02289419563021511, 0.02178645064122975, 0.02219472557771951, 0.021588515490293503, 0.0208683317177929, 0.019203111325623468, 0.015094081230927259, 0.012738844059640542, 0.010937373721390031, 0.009639891271945089, 0.008679873033543117, 0.007698492787312716, 0.0063614070968469605, 0.005839564517373219, 0.005443122397991829, 0.0045193517580628395, 0.003722919966094196, 0.0035805099178105593, 0.003556375791958999, 0.00315884810697753, 0.0031362239678855985, 0.00291520396785927, 0.0028207252180436626, 0.002649159541761037, 0.0028791238510166295, 0.0028062978162779473, 0.0028111041174270213, 0.0024473524063068908, 0.002713761274208082]
validation_losses len 30
[0.02176060391124338, 0.02250105675077066, 0.020775048732757567, 0.02143783333711326, 0.02152558905479964, 0.019956396577035775, 0.016039384090108796, 0.015448603191762231, 0.011271974047122057, 0.009592137183062732, 0.00791329194442369, 0.007252320768893697

<Figure size 640x480 with 1 Axes>

# Save/Load Model (given to you)

In [15]:
def save(model, train_losses, validation_losses, save_dir):
    """
    saving model, train losses, and validation losses
    Args:
        model              - NN to be saved
        train_losses       - history of losses for training dataset
        validation_losses  - history of losses for validation dataset
        save_dir           - directory where to save the above
    """
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    files = glob.glob(save_dir + '*')
    for f in files:
        os.remove(f) 
    
    
    torch.save(model, save_dir + "/model.dat")
    
    train_losses_f = open(save_dir + "/train_losses.txt", "wt")
    train_losses_f.writelines( "%.3f\n" % item for item in train_losses)
    
    validation_losses_f = open(save_dir + "/validation_losses.txt", "wt")
    validation_losses_f.writelines( "%.3f\n" % item for item in validation_losses)

    return
   

def load(save_dir):
    """
    loading model, train losses, and validation losses
    Args:
       save_dir  - dir name from where to load 
    """
    
    model = torch.load(save_dir + "/model.dat") 
    
    train_losses_f = open(save_dir + "/train_losses.txt", "rt")
    train_losses   = train_losses_f.readlines()
    train_losses   = [float(num) for num in train_losses]
    
    validation_losses_f = open(save_dir + "/validation_losses.txt", "rt")
    validation_losses   = validation_losses_f.readlines()
    validation_losses   = [float(num) for num in validation_losses]
    
    return (model, train_losses, validation_losses)
   

# Example saving and loading

In [16]:
# Create a directory, for example "./saves_12/", where you place your saved models

save(model, train_losses, validation_losses, "./saves/")

model, train_losses, validation_losses = load("./saves/")

D:\Programs\Anaconda\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CircleNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# Paint circles on loader images (given to you)

In [17]:
def paint_loader_circles(model, loader, out_dir):
    """
    This fucntion receives a model, a loader and an output directory. 
    For each image in the loader it paints a circle that the model identifies. 
    The images are saved in the given out_dir diretory. 
    Args:
        model   - network for idneitfying circles
        loader  - input data to use 
        out_dir - ouptut directory name (e.g.: 'draws/'). If directory does not exist, it is created.
                  If it exists, its files are deleted.
    """

    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
                  # (dropout is set to zero)

    k = 0
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    files = glob.glob(out_dir + '*')
    for f in files:
        os.remove(f) 
  
        
    for data in loader:
        # get inputs
        inputs = (data['image']).to(device)
        labels = (data['labels']).to(device)  # not using 
        img_fnames = data['fname'] 
      
        
        # forward
        outputs = model(inputs.float())
        curr_batch_size = np.shape(outputs)[0]
        image_size = np.shape(inputs[0])  # image_size = [3, w, h]
        _, width, height = image_size
        assert (width == height)
        
        for i in range (curr_batch_size): 
            x0 = (outputs[i, 0].item()) * width
            y0 = (1-outputs[i, 1].item()) * height
            r  = outputs[i, 2].item() * width #assume width=height here. Otherwise, circle becomes ellipse
   
            fname = img_fnames[i]
            k+=1
            print (str(k) + ".   " + fname)

            img = Image.open(fname)
            draw = ImageDraw.Draw(img, 'RGBA')
    
            draw.ellipse((x0 - r, y0 - r, x0 + r ,y0 + r), fill=(160, 64, 0, 90), outline=None)
    
            img.save(out_dir + fname.split('/')[-1])
    
        
    model.train()  #back to default
    return

# Example of how to paint circles produced by model

In [18]:
# Painting circles on images from validation loader and placing them in directory "./validation/draw/". 
# Notice that if the painted circle is seen only partly, it means that it is not inside the 
# [0,1]x[0,1] "box", which is the domain considered.  This means that your model has siginificant error


paint_loader_circles(model, validation_loader, './validation/draw/')
#paint_loader_circles(model, test_loader, './test/draw/')

1.   ./validation//images/0000.png
2.   ./validation//images/0001.png
3.   ./validation//images/0002.png
4.   ./validation//images/0003.png
5.   ./validation//images/0004.png
6.   ./validation//images/0005.png
7.   ./validation//images/0006.png
8.   ./validation//images/0007.png
9.   ./validation//images/0008.png
10.   ./validation//images/0009.png
11.   ./validation//images/0010.png
12.   ./validation//images/0011.png
13.   ./validation//images/0012.png
14.   ./validation//images/0013.png
15.   ./validation//images/0014.png
16.   ./validation//images/0015.png
17.   ./validation//images/0016.png
18.   ./validation//images/0017.png
19.   ./validation//images/0018.png
20.   ./validation//images/0019.png
21.   ./validation//images/0020.png
22.   ./validation//images/0021.png
23.   ./validation//images/0022.png
24.   ./validation//images/0023.png
25.   ./validation//images/0024.png
26.   ./validation//images/0025.png
27.   ./validation//images/0026.png
28.   ./validation//images/0027.png
2